In [1]:
import xml.etree.cElementTree as ET
import os
import re
import pandas as pd
import json
import random
random.seed(0)

import warnings
warnings.filterwarnings('ignore')

In [2]:
# tag_mapper = {'nu':'num', 'v':'vblex', 'cnj':'cnjcoo','3':'p3','2':'p2','1':'p1','':'None','s':'subj','p':'obj','a':'subj'}
tag_mapper = {'nu':'num', 'v':'vblex', 'cnj':'cnjcoo'}
NER_LIST = ['pn', 'on', 'fn', 'mn', 'sn', 'dn', 'gn', 'tn', 'wn', 'en', 'an', 'rn']

In [3]:
def txt_file_read(filename):
    lines=[]
    with open(filename, "r") as f:
        for line in f:
            line=line.strip()
            lines.append(line)
    return lines

In [4]:
def json_file_read(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

In [5]:

# def process_tags(tags_string):
#     mapped_tags = []
#     org_tags = []
#     for tg in tags_string.split('.'):
#         l_tag = tg.lower().replace('3','p3').replace('2','p2').replace('1','p1')
#         # splitting on "-" as well
#         for t in l_tag.split('-'):
#             mp_tag = t
#             if t in tag_mapper.keys():
#                 mp_tag = tag_mapper[t]
#             mapped_tags.append(mp_tag)
#     return mapped_tags

In [6]:
def process_row(row):
    orignal = row[1]
    segm = row[2]
    lem = "_"
    eng_translation = "_"
    # match pattern - all groups starting with word and containing square bracket
    w_group = re.findall(r"[\'@\(\)a-zA-Z0-9_-]+\[\w+\]",segm)
    for w in w_group:
        start_index = w.find('[')
        end_index = w.find(']')
        lem = w[:start_index]
        eng_translation = w[start_index+1:end_index]
        break
    
    if eng_translation=='1':
        eng_translation = lem

    # process and save the tags 
    # org_tag_spl = re.split("\.|\-",row[3])
    org_tag_spl = re.split("\.",row[3])
    mp_tags_spl = []
    ner_tag=False
    for t in org_tag_spl:
        t=t.lower()
        mp_tag = t
        if t in tag_mapper.keys():
            mp_tag = tag_mapper[t]
        if t in NER_LIST:
            ner_tag=True
        mp_tags_spl.append(mp_tag)
    if ner_tag:
        mp_tags_spl.append('np')

    return lem,eng_translation,org_tag_spl, mp_tags_spl

In [7]:
# s = 'N.DEM2.ABS'
# org_tag_spl = re.split("\.|\-",s)
# org_tag_spl

# MTAAC GOLD CORPUS 

In [8]:
# define dataframe
column = ['filename','id','form','seg','xpostag','lem','eng_translation','org_tags_split', 'processed_tags_split']
extracted_df = pd.DataFrame(columns = column)
extracted_df.head()

,filename,id,form,seg,xpostag,lem,eng_translation,org_tags_split,processed_tags_split


In [9]:
base_dir = 'data/to_dict/'
conll_files = os.listdir('data/to_dict/')

In [10]:
for conll_file in conll_files:
    temp = os.path.join(base_dir,conll_file)
    f = txt_file_read(temp)
    f_lines = [line.split("\t") for line in f]

    # start processing file
    processed_ls = []
    for row in f_lines[2:]:
        if len(row)>3:
            lem,eng_translation,org_tag_spl, mp_tags_spl = process_row(row)
            processed_ls.append([conll_file, row[0], row[1], row[2], row[3], lem,eng_translation,org_tag_spl, mp_tags_spl])

        # print(orignal, lem,eng_translation,tags)
        # if orignal!="_" and lem!="_":
        
    processed_df = pd.DataFrame(processed_ls, columns=column)
    # print(processed_df)
    extracted_df = extracted_df.append(processed_df)

In [11]:
# for conll_file in conll_files:
#     temp = os.path.join(base_dir,conll_file)
#     f = txt_file_read(temp)
#     f_lines = [line.split("\t") for line in f]

#     # start processing file
#     processed_ls = []
#     for row in f_lines:
#         if len(row) > 3:
#             orignal, lem, eng_translation,org_tags, processed_tags = process_row(row)
#             # print(orignal, lem,eng_translation,tags)
#             if orignal!="_" and lem!="_":
#                 processed_ls.append([conll_file, row[0],orignal, lem,eng_translation,org_tags, processed_tags ])
        
#     processed_df = pd.DataFrame(processed_ls, columns=column)
#     # print(processed_df)
#     extracted_df = extracted_df.append(processed_df)

In [12]:
extracted_df

,filename,id,form,seg,xpostag,lem,eng_translation,org_tags_split,processed_tags_split
0,P100723.conll,o.1.1,1(disz),1(disz)[one],NU,1(disz),one,[NU],[num]
1,P100723.conll,o.1.2,udu-nita2,udunita[ram][-ø],N.ABS,udunita,ram,"[N, ABS]","[n, abs]"
2,P100723.conll,o.1.3,ba-usz2,ba-usz[die][-ø],MID.V.3-SG-S,ba-usz,die,"[MID, V, 3-SG-S]","[mid, vblex, 3-sg-s]"
3,P100723.conll,o.2.1,1(disz),1(disz)[one],NU,1(disz),one,[NU],[num]
4,P100723.conll,o.2.2,sila4,sila[lamb],N,sila,lamb,[N],[n]
...,...,...,...,...,...,...,...,...,...
27,P480072.conll,s1.col1.4.3,limmu2-ba,limmu[four]-bi-ak[-ø],NU.3-SG-NH-POSS.GEN.ABS,limmu,four,"[NU, 3-SG-NH-POSS, GEN, ABS]","[num, 3-sg-nh-poss, gen, abs]"
28,P480072.conll,s1.col2.1.1,a-a-kal-la,Ayakala[1],PN,Ayakala,Ayakala,[PN],"[pn, np]"
29,P480072.conll,s1.col2.2.1,ensi2,ensik[ruler],N,ensik,ruler,[N],[n]
30,P480072.conll,s1.col2.3.1,umma{ki},Umma[1][-ak][-ø],SN.GEN.ABS,Umma,Umma,"[SN, GEN, ABS]","[sn, gen, abs, np]"


In [13]:
extracted_df.to_csv(r'mtaac_vocab_all.txt', index=None, sep=' ', mode='w')

In [14]:
all_tags= set()
for i in extracted_df.processed_tags_split:
    for j in i:
        all_tags.add(j)

In [15]:
all_tags

{'',
 '1-sg-poss',
 '2-sg-a',
 '2-sg-poss',
 '3-nh',
 '3-pl',
 '3-pl-dat',
 '3-pl-s',
 '3-sg-a',
 '3-sg-cop',
 '3-sg-dat',
 '3-sg-h',
 '3-sg-h-a',
 '3-sg-h-dat',
 '3-sg-h-poss',
 '3-sg-hn-p',
 '3-sg-nh',
 '3-sg-nh-a',
 '3-sg-nh-l3',
 '3-sg-nh-p',
 '3-sg-nh-poss',
 '3-sg-p',
 '3-sg-s',
 '_',
 'abl',
 'abs',
 'an',
 'ant',
 'cnjcoo',
 'com',
 'cop-3-pl',
 'cop-3-sg',
 'dah-h',
 'dat',
 'dat-h',
 'dat-nh',
 'dem2',
 'dn',
 'dub[tablet]-ak',
 'en',
 'erg',
 'f',
 'fin',
 'fn',
 'gen',
 'gen-abl',
 'gn',
 'l1',
 'l1-syn',
 'l2',
 'l2-nh',
 'l3',
 'l3-nh',
 'mid',
 'mn',
 'mod',
 'n',
 'neg',
 'nf',
 'np',
 'num',
 'on',
 'pf',
 'pl',
 'pn',
 'pt',
 'rdp',
 'rn',
 'sn',
 'sub',
 'term',
 'tn',
 'vblex',
 'ven',
 'wn'}

### all tags for sux_eng dict

In [16]:
#Sample-  <sdef n="n" 	c="Noun"/>
for tg in all_tags:
    tg_str = f'''<sdef n="{tg}" c="{tg}"/>'''
    print(tg_str)

<sdef n="neg" c="neg"/>
<sdef n="en" c="en"/>
<sdef n="2-sg-a" c="2-sg-a"/>
<sdef n="rdp" c="rdp"/>
<sdef n="nf" c="nf"/>
<sdef n="" c=""/>
<sdef n="cop-3-pl" c="cop-3-pl"/>
<sdef n="3-sg-h-dat" c="3-sg-h-dat"/>
<sdef n="np" c="np"/>
<sdef n="3-sg-h-a" c="3-sg-h-a"/>
<sdef n="cop-3-sg" c="cop-3-sg"/>
<sdef n="1-sg-poss" c="1-sg-poss"/>
<sdef n="dat" c="dat"/>
<sdef n="pt" c="pt"/>
<sdef n="3-pl-dat" c="3-pl-dat"/>
<sdef n="2-sg-poss" c="2-sg-poss"/>
<sdef n="3-pl-s" c="3-pl-s"/>
<sdef n="fn" c="fn"/>
<sdef n="cnjcoo" c="cnjcoo"/>
<sdef n="l1-syn" c="l1-syn"/>
<sdef n="dah-h" c="dah-h"/>
<sdef n="on" c="on"/>
<sdef n="gen-abl" c="gen-abl"/>
<sdef n="sn" c="sn"/>
<sdef n="pn" c="pn"/>
<sdef n="tn" c="tn"/>
<sdef n="mid" c="mid"/>
<sdef n="f" c="f"/>
<sdef n="3-sg-hn-p" c="3-sg-hn-p"/>
<sdef n="l1" c="l1"/>
<sdef n="mod" c="mod"/>
<sdef n="3-sg-nh-l3" c="3-sg-nh-l3"/>
<sdef n="dat-nh" c="dat-nh"/>
<sdef n="erg" c="erg"/>
<sdef n="3-sg-nh-p" c="3-sg-nh-p"/>
<sdef n="3-nh" c="3-nh"/>
<sdef 

## MAJOR POS TAGS
### np(Proper Name), num(numbers), cnjcoo, n, vblex

In [17]:
extracted_df.head(10)

,filename,id,form,seg,xpostag,lem,eng_translation,org_tags_split,processed_tags_split
0,P100723.conll,o.1.1,1(disz),1(disz)[one],NU,1(disz),one,[NU],[num]
1,P100723.conll,o.1.2,udu-nita2,udunita[ram][-ø],N.ABS,udunita,ram,"[N, ABS]","[n, abs]"
2,P100723.conll,o.1.3,ba-usz2,ba-usz[die][-ø],MID.V.3-SG-S,ba-usz,die,"[MID, V, 3-SG-S]","[mid, vblex, 3-sg-s]"
3,P100723.conll,o.2.1,1(disz),1(disz)[one],NU,1(disz),one,[NU],[num]
4,P100723.conll,o.2.2,sila4,sila[lamb],N,sila,lamb,[N],[n]
5,P100723.conll,o.2.3,ASZ-ur4,azur[type_of_sheep],N,azur,type_of_sheep,[N],[n]
6,P100723.conll,o.2.4,ba-usz2,ba-usz[die][-ø],MID.V.3-SG-S,ba-usz,die,"[MID, V, 3-SG-S]","[mid, vblex, 3-sg-s]"
7,P100723.conll,_,_,_,_,_,_,[_],[_]
8,P100723.conll,o.3.1,ki,ki[place],N,ki,place,[N],[n]
9,P100723.conll,o.3.2,ur-u3-ta,Uru[1][-ak]-ta,PN.GEN.ABL,Uru,Uru,"[PN, GEN, ABL]","[pn, gen, abl, np]"


# Named Entity

In [18]:
NP_sux = set()
NP_dict = set()
for i, row in extracted_df.iterrows():
    p_tag = row['processed_tags_split']
    #form = row['form']
    form = row['form'].replace(':','-').replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|') # to escape the special meaning of brackets
    lemma = row['lem']
    if 'np' in p_tag:
        remainig_tags = list(set(p_tag)-set(['np']))
        pos_tag = 'np'
        tg_str = '<np>'
        for t in remainig_tags:
            tg_str+='<'+t+'>'

        # for morph dict
        sux_morph = lemma+tg_str+':'+form

        # for translation dict
        sux_eng_dict = f'''<e><p><l>{lemma}<s n="{pos_tag}"/></l><r>{lemma}<s n="{pos_tag}"/></r></p></e>'''
        
        NP_sux.add(sux_morph)
        NP_dict.add(sux_eng_dict)

    

    # for j in NER_LIST:
    #     if j in p_tag:
    #         tag_found=True
    #         break
    # if tag_found:
    #     PN_LIST.add(row['form'])

In [19]:
for i in NP_sux:
    print(i)

Lugalszudari<np><pn><dat-h>:lugal-szu-da-ri2
Agugu<np><gen><pn>:a-gu-gu
Lukala<np><gen><pn><erg>:lu2-kal-la
Szarazame<np><gen><pn>:\{d\}szara2-za-me
Anszan<np><gn><abs>:an-sza\{ki\}
Nimgirdine<np><pn>:nimgir-di-ne
Lugalnimah<np><pn><dat-h>:lugal-ni-mah
Lubanda<np><pn><dat-h>:lu2-banda3\{da\}
Lu'inanna<np><gen><pn><abl>:lu2-\{d\}inanna-ta
Ninlil<np><dn><dat-h>:\{d\}nin-lil2
Kudamu<np><gen><pn>:ku5-da-mu
Dingirga'isa<np><pn><dat-h>:dingir-ga2-i3-sa6
Ga'esz<np><gen><sn><abs>:ga-esz\{ki\}
Nanna<np><dn><dat-h>:\{d\}nanna
Nesag<np><mn><l1>:nesag
Kinunir<np><gen><l1><sn>:ki-nu-nir\{ki\}
Puzurenlil<np><pn><dat-h>:puzur4-\{d\}en-lil2
Layasa<np><gen><pn>:la-a-sa6
Lamamu<np><pn><dat-h>:la-ma-mu
Libursimti<np><pn>:li-bur-si2-im-ti
E'abani<np><pn><dat-h>:e2-a-ba-ni
GANmasz<np><mn><l1>:GAN2-masz
Ningeszkimzi<np><gen><pn>:nin-geszkim-zi
Ilisa<np><pn>:i3-li2-sa6
engar<np><gen><pn><abl>:engar-ta
Ningina<np><pn>:nin9-gi-na
Urim<np><gen><sn>:uri5\{ki\}-ma
Biduga<np><gen><pn>:bi2-du11-ga
Szu'abba<np><gen>

In [20]:
for i in NP_dict:
    print(i)

<e><p><l>Erreb<s n="np"/></l><r>Erreb<s n="np"/></r></p></e>
<e><p><l>Gaduszana'um<s n="np"/></l><r>Gaduszana'um<s n="np"/></r></p></e>
<e><p><l>Lunigin<s n="np"/></l><r>Lunigin<s n="np"/></r></p></e>
<e><p><l>Urasznan<s n="np"/></l><r>Urasznan<s n="np"/></r></p></e>
<e><p><l>Urninmuga<s n="np"/></l><r>Urninmuga<s n="np"/></r></p></e>
<e><p><l>Lugina<s n="np"/></l><r>Lugina<s n="np"/></r></p></e>
<e><p><l>Szaszszurum<s n="np"/></l><r>Szaszszurum<s n="np"/></r></p></e>
<e><p><l>Kas<s n="np"/></l><r>Kas<s n="np"/></r></p></e>
<e><p><l>Gadu<s n="np"/></l><r>Gadu<s n="np"/></r></p></e>
<e><p><l>Ursagamu<s n="np"/></l><r>Ursagamu<s n="np"/></r></p></e>
<e><p><l>Kazallu<s n="np"/></l><r>Kazallu<s n="np"/></r></p></e>
<e><p><l>Dagamu<s n="np"/></l><r>Dagamu<s n="np"/></r></p></e>
<e><p><l>Suenabuszu<s n="np"/></l><r>Suenabuszu<s n="np"/></r></p></e>
<e><p><l>Szeszkala<s n="np"/></l><r>Szeszkala<s n="np"/></r></p></e>
<e><p><l>Tummal<s n="np"/></l><r>Tummal<s n="np"/></r></p></e>
<e><p><l>Amak

In [21]:
# #sample - Dugrna<np>:du-gr\{d\}-na
# for tg in PN_LIST:
#     original = tg.replace('{','\{').replace('}','\}')
#     formatted_1 = re.sub('\{[a-z\.]*\}','',tg)
#     formatted_2 = re.sub('[0-9]+','',formatted_1)
#     formatted_3 = formatted_2.replace('-','').capitalize()
#     print(formatted_3+'<np>:'+original)

In [22]:
# # to generate for dict
# # <e><p><l>Inanna<s n="np"/></l><r>Inanna<s n="np"/></r></p></e>
# for tg in PN_LIST:
#     original = tg.replace('{','\{').replace('}','\}')
#     formatted_1 = re.sub('\{[a-z\.]*\}','',tg)
#     formatted_2 = re.sub('[0-9]+','',formatted_1)
#     formatted_3 = formatted_2.replace('-','').capitalize()
#     print(f'''<e><p><l>{formatted_3}<s n="np"/></l><r>{formatted_3}<s n="np"/></r></p></e>''')

# Numbers

In [23]:
NP_sux = set()
NP_dict = set()
for i, row in extracted_df.iterrows():
    p_tag = row['processed_tags_split']
    form = row['form'].replace(':','-').replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|') # to escape the special meaning of brackets
    # form = row['form'].replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]') # to escape the special meaning of brackets
    lemma = row['lem']
    eng_translation = row['eng_translation']
    if 'num' in p_tag:
        remainig_tags = list(set(p_tag)-set(['num']))
        pos_tag = 'num'
        tg_str = '<num>'
        for t in remainig_tags:
            tg_str+='<'+t+'>'

        # for morph dict
        sux_morph = lemma+tg_str+':'+form

        # for translation dict
        sux_eng_dict = f'''<e><p><l>{lemma}<s n="{pos_tag}"/></l><r>{eng_translation}<s n="{pos_tag}"/></r></p></e>'''
        
        NP_sux.add(sux_morph)
        NP_dict.add(sux_eng_dict)

    

    # for j in NER_LIST:
    #     if j in p_tag:
    #         tag_found=True
    #         break
    # if tag_found:
    #     PN_LIST.add(row['form'])

In [24]:
for i in NP_sux:
    print(i)

1(gesz'u)<num>:1\(gesz'u\)
8(asz)<num>:8\(asz\)
4(disz)<num><abs><cop-3-sg>:4\(disz\)-am3
1(disz)<num><abl>:1\(disz\)-ta
8(disz)<num><term>:8\(disz\)-sze3
1(u)<num>:1\(u\)
4(u)<num><gen><cop-3-sg>:4\(u\)-kam
7(disz)<num><term>:7\(disz\)
3(disz)<num>:2/3\(disz\)
9(disz)<num>:9\(disz\)
6(disz)<num><abs>:6\(disz\)
4(u)<num>:4\(u\)
3(disz)<num><gen><cop-3-sg>:1/3\(disz\)-kam
8(disz)<num>:8\(disz@v\)
3(disz)<num><abl>:1/3\(disz\)-ta
3(disz)<num><gen>:3\(disz\)
9(disz)<num><gen><cop-3-sg>:9\(disz\)-kam
1(disz)<num>:1\(disz\)
6(disz)<num>:6\(disz\)
2(disz)<num>:2\(disz\)
_(_)<num>:\[n\]
4(disz)<num>:4\(disz\)
6(disz)<num>:5/6\(disz\)
1(disz)<num><term>:1\(disz\)
_<num><abs>:8\(szar2\)
1(disz)<num><l1>:1\(asz\)-a
1(u)<num><gen><cop-3-sg>:1\(u\)-kam
2(disz@t)<num><gen><cop-3-sg>:2\(disz@t\)-kam
igi4(disz)<num><abs>:igi-4\(disz\)-gal2
3(u)<num><gen><cop-3-sg>:3\(u\)-kam
asz<num><term>:asz-sze3
5(disz)<num><gen><cop-3-sg>:5\(disz\)-kam
5(disz)<num><gen><abs><cop-3-sg>:5\(disz\)-kam
2(disz)<num><g

In [25]:
for i in NP_dict:
    print(i)

<e><p><l>a<s n="num"/></l><r>labor<s n="num"/></r></p></e>
<e><p><l>4(u)<s n="num"/></l><r>ten<s n="num"/></r></p></e>
<e><p><l>igi4(disz)<s n="num"/></l><r>one<s n="num"/></r></p></e>
<e><p><l>2(disz)<s n="num"/></l><r>one<s n="num"/></r></p></e>
<e><p><l>min<s n="num"/></l><r>two<s n="num"/></r></p></e>
<e><p><l>1(asz@c)<s n="num"/></l><r>one<s n="num"/></r></p></e>
<e><p><l>4(disz)<s n="num"/></l><r>one<s n="num"/></r></p></e>
<e><p><l>9(gesz)<s n="num"/></l><r>sixty<s n="num"/></r></p></e>
<e><p><l>geme<s n="num"/></l><r>female_worker<s n="num"/></r></p></e>
<e><p><l>2(asz)<s n="num"/></l><r>one<s n="num"/></r></p></e>
<e><p><l>4(asz)<s n="num"/></l><r>one<s n="num"/></r></p></e>
<e><p><l>limmu<s n="num"/></l><r>four<s n="num"/></r></p></e>
<e><p><l>2(u)<s n="num"/></l><r>ten<s n="num"/></r></p></e>
<e><p><l>3(u)<s n="num"/></l><r>ten<s n="num"/></r></p></e>
<e><p><l>7(gesz)<s n="num"/></l><r>sixty<s n="num"/></r></p></e>
<e><p><l>8(disz)<s n="num"/></l><r>one<s n="num"/></r></p></

# Cnjcoo

In [26]:
NP_sux = set()
NP_dict = set()
for i, row in extracted_df.iterrows():
    p_tag = row['processed_tags_split']
    form = row['form'].replace(':','-').replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|') # to escape the special meaning of brackets
    # form = row['form'].replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]') # to escape the special meaning of brackets
    lemma = row['lem']
    eng_translation = row['eng_translation']
    if 'cnjcoo' in p_tag:
        remainig_tags = list(set(p_tag)-set(['cnjcoo']))
        pos_tag = 'cnjcoo'
        tg_str = '<cnjcoo>'
        for t in remainig_tags:
            tg_str+='<'+t+'>'

        # for morph dict
        sux_morph = lemma+tg_str+':'+form

        # for translation dict
        sux_eng_dict = f'''<e><p><l>{lemma}<s n="{pos_tag}"/></l><r>{eng_translation}<s n="{pos_tag}"/></r></p></e>'''
        
        NP_sux.add(sux_morph)
        NP_dict.add(sux_eng_dict)

    

    # for j in NER_LIST:
    #     if j in p_tag:
    #         tag_found=True
    #         break
    # if tag_found:
    #     PN_LIST.add(row['form'])

In [27]:
for i in NP_sux:
    print(i)

tukumbi<cnjcoo>:tukum-bi
u<cnjcoo>:u3


In [28]:
for i in NP_dict:
    print(i)

<e><p><l>tukumbi<s n="cnjcoo"/></l><r>if<s n="cnjcoo"/></r></p></e>
<e><p><l>u<s n="cnjcoo"/></l><r>and<s n="cnjcoo"/></r></p></e>


In [47]:
import re

In [50]:
s= 'asd..sd34/'

'asd_sd34_'

# Noun

In [52]:
NP_sux = set()
NP_dict = set()
for i, row in extracted_df.iterrows():
    p_tag = row['processed_tags_split']
    form = row['form'].replace(':','-').replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|') # to escape the special meaning of brackets
    # form = row['form'].replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]') # to escape the special meaning of brackets
    # lemma = row['lem']
    lemma = re.sub('[^a-zA-Z0-9]+','_',row['form'])

    eng_translation = row['eng_translation']
    if 'n' in p_tag:
        remainig_tags = list(set(p_tag)-set(['n']))
        pos_tag = 'n'
        tg_str = '<n>'
        for t in remainig_tags:
            tg_str+='<'+t+'>'

        # for morph dict
        sux_morph = lemma+tg_str+':'+form

        # for translation dict
        sux_eng_dict = f'''<e><p><l>{lemma}<s n="{pos_tag}"/></l><r>{eng_translation}<s n="{pos_tag}"/></r></p></e>'''
        
        NP_sux.add(sux_morph)
        NP_dict.add(sux_eng_dict)

    

    # for j in NER_LIST:
    #     if j in p_tag:
    #         tag_found=True
    #         break
    # if tag_found:
    #     PN_LIST.add(row['form'])

In [53]:
for i in NP_sux:
    print(i)

geme2_mu<n><1-sg-poss><abs>:geme2-mu
4_barig_<n><abs>:4\(barig\)
_gesz_an_za_am<n>:\{gesz\}an-za-am
sukkal<n><gen>:sukkal
gurusz_bi<n><3-sg-nh-poss><abs>:gurusz-bi
ga<n><abs>:ga
ka_gir_ra<n>:ka-gir-ra
udu<n>:udu
ur5_sze3<n><term>:ur5-sze3
gu4_gesz<n>:gu4-gesz
6_bur3_<n>:6\(bur3\)
nig2_gur11<n>:nig2-gur11
gi_zi<n><gen>:gi-zi
numun<n>:numun
nig2_szuku<n>:nig2-szuku
masz2_gal<n>:masz2-gal
sa2_du11<n><gen>:sa2-du11
banda3_sze3<n><term>:banda3-sze3
ru_ba_tum_ka<n><gen><l1>:ru-ba-tum-ka
_gesz_gigir<n><term>:\{gesz\}gigir
ku6<n>:ku6
muszen_du3<n>:muszen-du3
a_sza3_ga<n><gen>:a-sza3-ga
gudu4<n>:gudu4
gi_du3<n><term>:gi-du3
nu_banda3_gu4<n><gen>:nu-banda3-gu4
kiszib3_ba<n>:kiszib3-ba
ka_SAG_guru7<n><abs>:ka\(SAG\)-guru7
ugula<n><abs>:ugula
3_bur3_<n>:3\(bur3\)
gi_musz<n>:gi-musz
ninda_ta<n><abl>:ninda-ta
e2_ba_an<n><abs>:e2-ba-an
ge6_ba_a<n>:ge6-ba-a
aga3<n>:aga3
u8<n>:u8
id_gur2<n>:id-gur2
4_iku_<n>:4\(iku\)
kir11<n><abs>:kir11
DUL3<n>:DUL3
erin2_bi<n><gen><3-sg-nh-poss><abs>:erin2-bi
2_iku_<n

In [54]:
for i in NP_dict:
    print(i)

<e><p><l>ka_guru7<s n="n"/></l><r>supervisor<s n="n"/></r></p></e>
<e><p><l>gab2_ra<s n="n"/></l><r>rider<s n="n"/></r></p></e>
<e><p><l>im_ku3_sig17<s n="n"/></l><r>orpiment<s n="n"/></r></p></e>
<e><p><l>ka_us2_sa2<s n="n"/></l><r>profession_type<s n="n"/></r></p></e>
<e><p><l>e2_gal_e<s n="n"/></l><r>palace<s n="n"/></r></p></e>
<e><p><l>titab2<s n="n"/></l><r>_<s n="n"/></r></p></e>
<e><p><l>ib2<s n="n"/></l><r>middle<s n="n"/></r></p></e>
<e><p><l>gaz<s n="n"/></l><r>type_of_garlic<s n="n"/></r></p></e>
<e><p><l>_ansze_kunga2<s n="n"/></l><r>equid_type<s n="n"/></r></p></e>
<e><p><l>1_barig_ta<s n="n"/></l><r>unit<s n="n"/></r></p></e>
<e><p><l>zi3_da_ka<s n="n"/></l><r>flour<s n="n"/></r></p></e>
<e><p><l>szu_i<s n="n"/></l><r>barber<s n="n"/></r></p></e>
<e><p><l>dida<s n="n"/></l><r>sweet_wort<s n="n"/></r></p></e>
<e><p><l>numun_na<s n="n"/></l><r>seed<s n="n"/></r></p></e>
<e><p><l>a_ra<s n="n"/></l><r>part_of_boat<s n="n"/></r></p></e>
<e><p><l>a_sza3<s n="n"/></l><r>field<s

In [55]:
# OLD Nouns - 
# lugal<n>:lugal
# digir<n>:digir
# ama<n>:ama
# nin<n>:nin
# egal<n>:e2-gal
# dubsar<n>:dub-sar
# ursag<n>:ur-sag
# e2<n>:e2
# udu<n>:udu
# ur<n>:ur
# na<n>:na<4>
# dub-sar<n>:dub-sar
# ki<n>:ki
# di-ku5<n>:di-ku5
# ezem<n>:ezem


# Verb

In [43]:
NP_sux = set()
NP_dict = set()
for i, row in extracted_df.iterrows():
    p_tag = row['processed_tags_split']
    form = row['form'].replace(':','-').replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|') # to escape the special meaning of brackets
    # form = row['form'].replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]') # to escape the special meaning of brackets
    # lemma = row['lem']
    lemma = re.sub('[^a-zA-Z0-9]+','_',row['form'])

    eng_translation = row['eng_translation']
    if 'vblex' in p_tag:
        remainig_tags = list(set(p_tag)-set(['vblex']))
        pos_tag = 'vblex'
        tg_str = '<vblex>'
        for t in remainig_tags:
            tg_str+='<'+t+'>'

        # for morph dict
        sux_morph = lemma+tg_str+':'+form

        # for translation dict
        sux_eng_dict = f'''<e><p><l>{lemma}<s n="{pos_tag}"/></l><r>{eng_translation}<s n="{pos_tag}"/></r></p></e>'''
        
        NP_sux.add(sux_morph)
        NP_dict.add(sux_eng_dict)

    

    # for j in NER_LIST:
    #     if j in p_tag:
    #         tag_found=True
    #         break
    # if tag_found:
    #     PN_LIST.add(row['form'])

In [44]:
for i in NP_sux:
    print(i)

ba_du3_a<vblex><mid><3-sg-s>:ba-du3-a
niga<vblex><nf><abs>:niga
dabx_U8__ba<vblex><pt><nf>:dabx\(U8\)-ba
dagal<vblex><nf><abs>:dagal
tab_ba<vblex><pt><nf>:tab-ba
diri_ga_am3<vblex><pt><nf><abs><cop-3-sg>:diri-ga-am3
si_e3<vblex><nf><abs>:si-e3
gesz_gar_ra<vblex><pt><nf>:gesz-gar-ra
zi_ga_am3<vblex><pt><nf><abs><cop-3-sg>:zi-ga-am3
in_du3_a<vblex><3-sg-p><sub><3-sg-h-a><fin>:in-du3-a
gazx_KUM_<vblex><pt><nf>:gazx\(KUM\)
zi_ga<vblex><pt><nf><abs>:zi-ga
mu_hul_sze3<vblex><3-sg-p><ven><term><3-sg-h-a>:mu-hul-sze3
ba_a_gar<vblex><mid><l2><3-sg-s>:ba-a-gar
ra_a<vblex><pt><nf>:ra-a
us2_sa_bi<vblex><pt><nf><3-sg-nh-poss>:us2-sa-bi
si_ga<vblex><pt><nf>:si-ga
uru4_a<vblex><pt><nf>:uru4-a
kesz2_ra2<vblex><pt><nf>:kesz2-ra2
in_na_szum2_ma<vblex><l1><3-sg-h><3-sg-p><dat><3-sg-h-a><sub><fin>:in-na-szum2-ma
ku5_da<vblex><pt><nf>:ku5-da
szar3<vblex><nf><abs>:szar3
gazx_KUM_<vblex><nf><abs>:gazx\(KUM\)
sig15_ga<vblex><pt><nf>:sig15-ga
nu_la2<vblex><neg><3-sg-p><sub><3-sg-nh-a>:nu-la2
ba_kux_KWU147_<vbl

In [46]:
for i in NP_dict:
    print(i)

<e><p><l>ba_ne_tar_ra<s n="vblex"/></l><r>ask<s n="vblex"/></r></p></e>
<e><p><l>ga2_ga2<s n="vblex"/></l><r>place<s n="vblex"/></r></p></e>
<e><p><l>al_zi_ra<s n="vblex"/></l><r>break<s n="vblex"/></r></p></e>
<e><p><l>zi_ga<s n="vblex"/></l><r>issue<s n="vblex"/></r></p></e>
<e><p><l>zal_la<s n="vblex"/></l><r>pass<s n="vblex"/></r></p></e>
<e><p><l>si_ga<s n="vblex"/></l><r>load<s n="vblex"/></r></p></e>
<e><p><l>zi_ga<s n="vblex"/></l><r>raise<s n="vblex"/></r></p></e>
<e><p><l>si_ga<s n="vblex"/></l><r>line<s n="vblex"/></r></p></e>
<e><p><l>mu_ne_dim2<s n="vblex"/></l><r>create<s n="vblex"/></r></p></e>
<e><p><l>ba_ti<s n="vblex"/></l><r>accept<s n="vblex"/></r></p></e>
<e><p><l>ba_gen_na_a<s n="vblex"/></l><r>go<s n="vblex"/></r></p></e>
<e><p><l>sa10_am3<s n="vblex"/></l><r>pay_for<s n="vblex"/></r></p></e>
<e><p><l>saga<s n="vblex"/></l><r>good<s n="vblex"/></r></p></e>
<e><p><l>nu_gi4_gi4_da<s n="vblex"/></l><r>turn<s n="vblex"/></r></p></e>
<e><p><l>zi_ga<s n="vblex"/></l><r

# Extra

In [79]:
import re
s = '{d}nin-mar{ki}-ta'

In [80]:
re.sub('\{[a-z\.]*\}','',s)

'nin-mar-ta'

In [81]:
re.findall(r'\{.*\}',s)

['{d}nin-mar{ki}']

In [112]:
extracted_df.head()

,filename,id,form,lem,eng_translation,org_tags,processed_tags
0,P100723.conll,o.1.1,1(disz),1(disz),one,NU,[num]
1,P100723.conll,o.1.2,udu-nita2,udunita,ram,N.ABS,"[n, abs]"
2,P100723.conll,o.1.3,ba-usz2,ba-usz,die,MID.V.3-SG-S,"[mid, vblex, p3, sg, s]"
3,P100723.conll,o.2.1,1(disz),1(disz),one,NU,[num]
4,P100723.conll,o.2.2,sila4,sila,lamb,N,[n]


In [23]:
extracted_df

,filename,id,form,lem,eng_translation,org_tags,processed_tags
0,P100723.conll,o.1.1,1(disz),1(disz),one,NU,[num]
1,P100723.conll,o.1.2,udu-nita2,udunita,ram,N.ABS,"[n, abs]"
2,P100723.conll,o.1.3,ba-usz2,ba-usz,die,MID.V.3-SG-S,"[mid, vblex, p3, sg, s]"
3,P100723.conll,o.2.1,1(disz),1(disz),one,NU,[num]
4,P100723.conll,o.2.2,sila4,sila,lamb,N,[n]
...,...,...,...,...,...,...,...
27,P480072.conll,s1.col1.4.3,limmu2-ba,limmu,four,NU.3-SG-NH-POSS.GEN.ABS,"[num, p3, sg, nh, poss, gen, abs]"
28,P480072.conll,s1.col2.1.1,a-a-kal-la,Ayakala,1,PN,[pn]
29,P480072.conll,s1.col2.2.1,ensi2,ensik,ruler,N,[n]
30,P480072.conll,s1.col2.3.1,umma{ki},Umma,1,SN.GEN.ABS,"[sn, gen, abs]"


In [26]:
# extracted_df.to_csv('vocab.csv')                                            

In [27]:
# extracted_df.to_csv(r'vocab_form.txt', header=None, index=None, sep=' ', mode='a')

In [332]:
# converting all proper noun (FN, DN, PN) etc to there names
for i, row in extracted_df.iterrows():
    if row['eng_translation']=='1':
        row['eng_translation'] = row['lem']

In [392]:
extracted_df[['form','lem','eng_translation']]

,form,lem,eng_translation
0,1(disz),1(disz),one
1,udu-nita2,udunita,ram
2,ba-usz2,ba-usz,die
3,1(disz),1(disz),one
4,sila4,sila,lamb
...,...,...,...
27,limmu2-ba,limmu,four
28,a-a-kal-la,Ayakala,1
29,ensi2,ensik,ruler
30,umma{ki},Umma,1


## Alphabets

In [333]:
alpha = set()
for word in extracted_df['form']:
    for char in word:
        alpha.add(char)
alpha = sorted(alpha)
print(len(alpha))
alphs_string = "".join(alpha)
alphs_string

66


"#'()-./0123456789:<>?@ABDEGIKLMNPRSTUWZ[]abcdeghiklmnpqrstuvwxz{|}"

### Add alphabets to dict

In [334]:
base_dix_path = './apertium-sux.sux.dix'
mytree = ET.parse(base_dix_path)
root = mytree.getroot()
root

<Element 'dictionary' at 0x13b021bc0>

In [335]:
old_alpha = root.find('alphabet')
root.remove(old_alpha)

In [336]:
el_alpha = ET.Element("alphabet")
el_alpha.text = alphs_string

In [337]:
root.append(el_alpha)

In [338]:
tree = ET.ElementTree(root)
# ET.indent(tree, space="\t", level=0)

In [339]:
sux_dix_name = "apertium-sux.sux.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

## unique tags

In [340]:
uniqe_tags = set()
for tg in extracted_df['tags']:
    for t in tg:
        if t not in ['','_']:
            uniqe_tags.add(t)
uniqe_tags = sorted(uniqe_tags)
uniqe_tags[:5]

['a', 'abl', 'abs', 'an', 'ant']

In [341]:
len(uniqe_tags)

53

### mappings (once get the names)

In [342]:
# read json
filepath = '../Data/CDLI-CoNLL-to-CoNLLU-Converter/cdliconll2conllu/mapping.json'
cdliconll_map_data = json_file_read(filepath)

In [343]:
cdliconll_map_data

{'CDLI-CoNLL-Fields': ['ID',
  'FORM',
  'SEGM',
  'XPOSTAG',
  'HEAD',
  'DEPREL',
  'MISC'],
 'CoNLL-U-Fields': ['ID',
  'FORM',
  'LEMMA',
  'UPOSTAG',
  'XPOSTAG',
  'FEATS',
  'HEAD',
  'DEPREL',
  'DEPS',
  'MISC'],
 'xPOSTagMapping': {'AJ': 'ADJ',
  'AV': 'ADV',
  'NU': 'NUM',
  'CNJ': 'CCONJ',
  'DET': 'DET',
  'J': 'INTJ',
  'N': 'NOUN',
  'V': 'VERB',
  'PRP': 'ADP',
  'DN': 'PROPN',
  'EN': 'PROPN',
  'GN': 'PROPN',
  'MN': 'PROPN',
  'PN': 'PROPN',
  'RN': 'PROPN',
  'SN': 'PROPN',
  'TN': 'PROPN',
  'WN': 'PROPN',
  'AN': 'PROPN',
  'CN': 'PROPN',
  'FN': 'PROPN',
  'LN': 'PROPN',
  'ON': 'PROPN',
  'QN': 'PROPN',
  'YN': 'PROPN'},
 'posToFeatMapping': {'NOUN': ['Animacy', 'Case', 'Number'],
  'VERB': ['Aspect',
   'Mood',
   'Number',
   'Person',
   'Polarity',
   'Verb Form',
   'Voice'],
  'ADJ': ['Case', 'Number'],
  'NUM': ['Case'],
  'PROPN': ['Case', 'Number', 'Animacy']},
 'featureMapping': {'Case': {'ABS': 'Abs',
   'GEN': 'Gen',
   'ERG': 'Erg',
   'ABL': 'Abl'}

### adding tags to dict

In [344]:
# base_dix_path = './apertium-sux.sux.dix'
# mytree = ET.parse(base_dix_path)
# root = mytree.getroot()

In [345]:
old_alpha = root.find('sdefs')
root.remove(old_alpha)

In [346]:
symb_defs = ET.Element("sdefs")
root.append(symb_defs)
for tag in uniqe_tags:
    s1 = ET.SubElement(symb_defs,"sdef")
    s1.set('n',tag)
    # symb_defs.append(s1)

In [347]:
tree = ET.ElementTree(root)
ET.indent(tree, ' ')

In [348]:
sux_dix_name = "apertium-sux.sux.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

In [119]:
s = '''<prn><subj><p1><mf><sg>:-ga-e
<prn><subj><p1><mf><sg>:-ga2-e
<prn><subj><p1><mf><sg>:-ge26-e
<prn><subj><p2><mf><sg>:-za-e
<prn><subj><p2><mf><sg>:-ze2-e
<prn><subj><p3><mf><sg>:-e-ne
<prn><subj><p3><mf><sg>:-a-ne
<prn><subj><p3><nt><pl>:-e-ne-ne
<prn><subj><p3><nt><pl>:-a-ne-ne'''

In [128]:
temp = []
for i in s.split('\n'):
    temp.append(i.split(':')[1])

In [129]:
pd.DataFrame(temp)

,0
0,-ga-e
1,-ga2-e
2,-ge26-e
3,-za-e
4,-ze2-e
5,-e-ne
6,-a-ne
7,-e-ne-ne
8,-a-ne-ne


## Paradefs

In [349]:
para_defs = root.find('pardefs')

In [350]:
if para_defs:
    root.remove(para_defs)

In [351]:
extracted_sux_eng =  extracted_df[['filename','form','lem','eng_translation','tags']]
extracted_sux_eng.head()

,filename,form,lem,eng_translation,tags
0,P100723.conll,1(disz),1(disz),one,[num]
1,P100723.conll,udu-nita2,udunita,ram,"[n, abs]"
2,P100723.conll,ba-usz2,ba-usz,die,"[mid, vblex, p3, sg, s]"
3,P100723.conll,1(disz),1(disz),one,[num]
4,P100723.conll,sila4,sila,lamb,[n]


In [352]:
extracted_sux_eng.shape

(15394, 5)

In [353]:
extracted_sux_eng['tags_str'] = extracted_sux_eng['tags'].apply(lambda x:"_".join(x))

In [354]:
extracted_sux_eng

,filename,form,lem,eng_translation,tags,tags_str
0,P100723.conll,1(disz),1(disz),one,[num],num
1,P100723.conll,udu-nita2,udunita,ram,"[n, abs]",n_abs
2,P100723.conll,ba-usz2,ba-usz,die,"[mid, vblex, p3, sg, s]",mid_vblex_p3_sg_s
3,P100723.conll,1(disz),1(disz),one,[num],num
4,P100723.conll,sila4,sila,lamb,[n],n
...,...,...,...,...,...,...
27,P480072.conll,limmu2-ba,limmu,four,"[num, p3, sg, nh, poss, gen, abs]",num_p3_sg_nh_poss_gen_abs
28,P480072.conll,a-a-kal-la,Ayakala,Ayakala,[pn],pn
29,P480072.conll,ensi2,ensik,ruler,[n],n
30,P480072.conll,umma{ki},Umma,Umma,"[sn, gen, abs]",sn_gen_abs


In [355]:
# extracted_sux_eng_unique = extracted_sux_eng.drop_duplicates(subset=['form','tags_str'])
extracted_sux_eng_unique = extracted_sux_eng.drop_duplicates(subset=['form','tags_str'])
extracted_sux_eng_unique = extracted_sux_eng_unique[extracted_sux_eng_unique['tags_str']!='']
extracted_sux_eng_unique = extracted_sux_eng_unique[extracted_sux_eng_unique['tags_str']!='_']
extracted_sux_eng_unique.shape

(2918, 6)

In [356]:
extracted_sux_eng_unique_sort = extracted_sux_eng_unique.sort_values(['form']).reset_index(drop=True)
extracted_sux_eng_unique_sort

,filename,form,lem,eng_translation,tags,tags_str
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg
1,P142766.conll,1(asz),1(asz),one,[num],num
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num
4,P454156.conll,1(ban2),1(ban),unit,[n],n
...,...,...,...,...,...,...
2913,P122096.conll,{uruda}ha-bu3-da,habuda,hoe,[n],n
2914,P136937.conll,{uruda}ur2-zi-ir,urzir,tree_type,[n],n
2915,P416146.conll,{uruda}zi-ir,zir,slip,[n],n
2916,P478822.conll,|BAD3.AN|{ki}-ta,Der,Der,"[sn, abl]",sn_abl


In [357]:
extracted_sux_eng_unique_sort.tail(20)

,filename,form,lem,eng_translation,tags,tags_str
2898,P141530.conll,{gi}si2-im-tum,simtum,measuring_container,[n],n
2899,P144427.conll,{gi}su7-su7,susu,reed_type,[n],n
2900,P141675.conll,{kusz}du10-gan,duggan,leather_bag,[n],n
2901,P203987.conll,{kusz}du10-gan-na,duggan,leather_bag,"[n, gen, lp1]",n_gen_lp1
2902,P141675.conll,{kusz}e-sir2,esir,shoe,[n],n
2903,P142827.conll,{kusz}suhub2,suhub,boots,[n],n
2904,P100958.conll,{kusz}suhub2-sze3,suhub,boots,"[n, term]",n_term
2905,P141675.conll,{kusz}szu4-gan,szugan,pot_type,[n],n
2906,P141675.conll,{kusz}ummu3,ummud,waterskin,[n],n
2907,P101761.conll,{munus}asz2-gar3,aszgar,female_kid,[n],n


In [358]:
main_tags = ['n', 'vblex', 'aj', 'av', 'num', 'cnjcoo', 'det', 'j', 'pp']
NER_LIST = ['pn', 'on', 'fn', 'mn', 'sn', 'dn', 'gn', 'tn', 'wn', 'en', 'an', 'rn']
# tag_mapper = {'nu':'num', 'v':'vblex', 'cnj':'cnjcoo'}

In [359]:
primary_tag = []
secondary_tags = []

for i, row in extracted_sux_eng_unique_sort.iterrows():
    p_tg=''
    for j in main_tags:
        if j in row['tags']:
            p_tg = j
            break
    if p_tg=='' and len(row['tags'])>0:
        p_tg = row['tags'][0]
        if p_tg in NER_LIST:
            p_tg = 'n'

    
    secondary_tag_ls = list(set(row['tags'])-set([p_tg]))
    
    # add tags
    primary_tag.append(p_tg)
    secondary_tags.append(secondary_tag_ls)


In [360]:
extracted_sux_eng_unique_sort['primary_tag'] = primary_tag
extracted_sux_eng_unique_sort['secondary_tags'] = secondary_tags
extracted_sux_eng_unique_sort.head()

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]


In [361]:
extracted_sux_eng_unique_sort.tail(20)

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
2898,P141530.conll,{gi}si2-im-tum,simtum,measuring_container,[n],n,n,[]
2899,P144427.conll,{gi}su7-su7,susu,reed_type,[n],n,n,[]
2900,P141675.conll,{kusz}du10-gan,duggan,leather_bag,[n],n,n,[]
2901,P203987.conll,{kusz}du10-gan-na,duggan,leather_bag,"[n, gen, lp1]",n_gen_lp1,n,"[gen, lp1]"
2902,P141675.conll,{kusz}e-sir2,esir,shoe,[n],n,n,[]
2903,P142827.conll,{kusz}suhub2,suhub,boots,[n],n,n,[]
2904,P100958.conll,{kusz}suhub2-sze3,suhub,boots,"[n, term]",n_term,n,[term]
2905,P141675.conll,{kusz}szu4-gan,szugan,pot_type,[n],n,n,[]
2906,P141675.conll,{kusz}ummu3,ummud,waterskin,[n],n,n,[]
2907,P101761.conll,{munus}asz2-gar3,aszgar,female_kid,[n],n,n,[]


In [362]:
extracted_sux_eng_unique_sort['primary_tag'].unique()

array(['num', 'n', 'vblex', 'cnjcoo'], dtype=object)

In [363]:
# tree = ET.ElementTree(root)
# ET.indent(tree, ' ')

In [364]:
# sux_dix_name = "apertium-sux.sux.dix"
# with open (sux_dix_name, "wb") as f :
#         tree.write(f,encoding = "UTF-8", xml_declaration = True)

## Setions

In [365]:
secs = root.findall('section')

In [366]:
for sec in secs:
    root.remove(sec)

In [367]:
extracted_sux_eng_unique_sort.head()

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]


In [368]:
# tag_mapper = {'N':'n','NU':'num', 'V':'vblex', 'CNJ':'cnjcoo'}

In [369]:
section = ET.Element("section")
section.set('id',"main")
section.set('type',"standard")
root.append(section)

In [370]:
for i,row in extracted_sux_eng_unique_sort.iterrows():
    surface_form = row['form']
    lem = row['lem']
    tags = row['tags']
    p_tag = row['primary_tag']
    
    e = ET.SubElement(section,"e")
    p = ET.SubElement(e,"p")
    
    # left
    l = ET.SubElement(p,"l")
    l.text = surface_form
    
    # right
    r = ET.SubElement(p,"r")
    r.text = lem
    s = ET.SubElement(r,"s")
    if p_tag in uniqe_tags:
        s.set("n",p_tag)
    else:
        s.set("n","n")
        
    # for t in tags:
    #     if t in uniqe_tags:
    #         s = ET.SubElement(r,"s")
    #         mp_tag = t
    #         s.set("n",mp_tag)
    #     # r.append(s)

In [371]:
# uniqe_tags

In [372]:
extracted_sux_eng_unique_sort

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]
...,...,...,...,...,...,...,...,...
2913,P122096.conll,{uruda}ha-bu3-da,habuda,hoe,[n],n,n,[]
2914,P136937.conll,{uruda}ur2-zi-ir,urzir,tree_type,[n],n,n,[]
2915,P416146.conll,{uruda}zi-ir,zir,slip,[n],n,n,[]
2916,P478822.conll,|BAD3.AN|{ki}-ta,Der,Der,"[sn, abl]",sn_abl,n,"[sn, abl]"


In [373]:
tree = ET.ElementTree(root)
# ET.indent(tree, ' ')

In [374]:
tree_string_sux = ET.tostring(root,xml_declaration=True).decode("utf-8").replace('<e>','\n<e>').encode('utf-8')

In [375]:
# temp.encode('utf-8')

In [377]:
sux_dix_name = "apertium-sux.sux.dix"
with open (sux_dix_name, "wb") as f :
        f.write(tree_string_sux)

In [376]:
# sux_dix_name = "apertium-sux.sux.dix"
# with open (sux_dix_name, "wb") as f :
#         tree.write(f,encoding = "UTF-8", xml_declaration = True)

# Bi-Dictionary

In [222]:
extracted_sux_eng_unique_sort

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]
...,...,...,...,...,...,...,...,...
2913,P122096.conll,{uruda}ha-bu3-da,habuda,hoe,[n],n,n,[]
2914,P136937.conll,{uruda}ur2-zi-ir,urzir,tree_type,[n],n,n,[]
2915,P416146.conll,{uruda}zi-ir,zir,slip,[n],n,n,[]
2916,P478822.conll,|BAD3.AN|{ki}-ta,Der,Der,"[sn, abl]",sn_abl,n,"[sn, abl]"


In [240]:
sum_eng_dict_v1 = {}
for sux_lem,eng_trans in zip(extracted_df['lem'],extracted_df['eng_translation']):
    sum_eng_dict_v1[sux_lem] = eng_trans
print(len(sum_eng_dict_v1))
sum_eng_dict_v1

1679


{'1(disz)': 'one',
 'udunita': 'ram',
 'ba-usz': 'die',
 'sila': 'unit',
 'azur': 'type_of_sheep',
 'ki': 'place',
 'Uru': 'Uru',
 'szag': 'heart',
 'bala': 'turn',
 'kiszib': 'seal',
 'Arad': 'Arad',
 'iti': 'month',
 'Szesagku': 'Szesagku',
 'mu': 'year',
 'us': 'follow',
 'Kimasz': 'Kimasz',
 'ba-hulu': 'destroy',
 'Aradmu': 'Aradmu',
 'dubsar': 'scribe',
 'dumu': 'child',
 'Urnigar': 'Urnigar',
 'kusz': 'leather',
 'udu': 'sheep',
 'u': 'grass',
 'maszgal': 'billy_goat',
 'ba-ug': 'die',
 'ud': 'day',
 '1(u)': 'ten',
 'la': 'hang',
 '1(disz@t)': 'one',
 'Tummal': 'Tummal',
 'Endingirmu': 'Endingirmu',
 "Szulgi'irimu": "Szulgi'irimu",
 'szu': 'hand',
 '-teg': 'accept',
 'Akiti': 'Akiti',
 'en': 'priest',
 'Nanna': 'Nanna',
 'ba-hug': 'hire',
 '3(disz)': 'one',
 'gig': 'unit',
 '2(u)': 'ten',
 'sze': 'barley',
 'igi6(disz)': 'one',
 'Urbagara': 'Urbagara',
 '1(ban)': 'unit',
 'inun': 'ghee',
 '6(disz)': 'one',
 'ga': 'milk',
 'gazi': 'condiment',
 'Urmes': 'Urmes',
 'Urdumuzi': 'Urdu

In [284]:
base_dix_path = '../apertium-sux-eng/apertium-sux-eng.sux-eng.dix'
mytree = ET.parse(base_dix_path)
root = mytree.getroot()
root

<Element 'dictionary' at 0x10c38c450>

### adding tags to dict

In [285]:
# base_dix_path = './apertium-sux.sux.dix'
# mytree = ET.parse(base_dix_path)
# root = mytree.getroot()

In [286]:
old_alpha = root.find('sdefs')
root.remove(old_alpha)

In [287]:
symb_defs = ET.Element("sdefs")
root.append(symb_defs)
for tag in uniqe_tags:
    s1 = ET.SubElement(symb_defs,"sdef")
    s1.set('n',tag)
    # symb_defs.append(s1)

In [288]:
tree = ET.ElementTree(root)
ET.indent(tree, ' ')

In [289]:
# sux_dix_name = "apertium-sux.sux.dix"
# with open (sux_dix_name, "wb") as f :
#         tree.write(f,encoding = "UTF-8", xml_declaration = True)

In [290]:
# old_alpha = root.find('alphabet')
# root.remove(old_alpha)

In [291]:
# el_alpha = ET.Element("alphabet")
# el_alpha.text = alphs_string

In [292]:
# root.append(el_alpha)

In [293]:
# tree = ET.ElementTree(root)
# # ET.indent(tree, space="\t", level=0)

## ==========================

## Sections

In [294]:
secs = root.findall('section')
for sec in secs:
    root.remove(sec)

In [295]:
section = ET.Element("section")
section.set('id',"main")
section.set('type',"standard")
root.append(section)

In [296]:
for i,row in extracted_sux_eng_unique_sort.iterrows():
    surface_form = row['form']
    lem = row['lem']
    tags = row['tags']
    eng_translation = row['eng_translation']
    p_tag = row['primary_tag']
    
    e = ET.SubElement(section,"e")
    p = ET.SubElement(e,"p")
    
    # left
    l = ET.SubElement(p,"l")
    l.text = lem
    s = ET.SubElement(l,"s")
    if p_tag in uniqe_tags:
        s.set("n",p_tag)
    else:
        s.set("n","n")



    # for t in tags:
    #     if t in uniqe_tags:
    #         s = ET.SubElement(l,"s")
    #         mp_tag = t
    #         try:
    #             mp_tag = tag_mapper[t]
    #         except:
    #             pass
    #         s.set("n",mp_tag)


    # right
    r = ET.SubElement(p,"r")
    r.text = eng_translation
    s = ET.SubElement(r,"s")
    if p_tag in uniqe_tags:
        s.set("n",p_tag)
    else:
        s.set("n","n")




    # for t in tags:

    #     mp_tag = t
    #     try:
    #         mp_tag = tag_mapper[t]
    #     except:
    #         pass
    # s.set("n","n")
        # r.append(s)

In [297]:
tree = ET.ElementTree(root)
# ET.indent(tree, ' ')

In [298]:
sux_dix_name = "../apertium-sux-eng/apertium-sux-eng.sux-eng.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

In [317]:
temp = ET.tostring(root,xml_declaration=True).decode("utf-8").replace('<e>','\n<e>').encode('utf-8')

In [318]:
# temp.encode('utf-8')

In [319]:
sux_dix_name = "../apertium-sux-eng/apertium-sux-eng.sux-eng.dix"
with open (sux_dix_name, "wb") as f :
        f.write(temp)